In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, StackingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours

In [2]:
#importing files
train = pd.read_csv('/run/media/devesh/Hard Disk/Projects/Machinehack-Melanoma Tumor Size Predicition/input/Train.csv')
test = pd.read_csv('/run/media/devesh/Hard Disk/Projects/Machinehack-Melanoma Tumor Size Predicition/input/Test.csv')

In [3]:
train.head()

,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,tumor_size
0,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.487,2758.76,72,39.3620,14.103
1,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.881,5952.53,240,22.0253,2.648
2,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.093,4635.26,73,29.9963,1.688
3,13093.80,4510.06,0.34444,155.4400,1.812195e+06,173.015,5273.87,32,28.1354,3.796
4,7545.21,2882.36,0.38201,85.1237,1.043918e+06,124.414,3263.35,57,35.0200,18.023


In [4]:
target = train.pop('tumor_size')

In [5]:
# Feature engineering

train['penalty-err'] = train['malign_penalty'] - train['err_malign']
train['d_size-ratio'] = train['damage_size'] - train['damage_ratio']
train['d_ratio-m_ratio'] = train['damage_ratio'] - (train['malign_ratio'])
train['penalty/std'] = train['malign_penalty']/train['std_dev_malign']
train['mass/area'] = (train['mass_npea'])/(train['exposed_area'])
train['area/mass'] = train['exposed_area']/train['mass_npea']
train['err/std'] = train['penalty-err']/train['std_dev_malign']
train['dsr/ps'] = train['damage_size']/train['penalty-err']
train['std/area'] = train['std_dev_malign']/train['exposed_area']
train['err/area'] = train['err_malign']/train['exposed_area']
train['dr/area'] = (train['damage_ratio']*100)/train['exposed_area']
train['std/err'] = (train['std_dev_malign']+1)/(train['err_malign']+1)
train['penalty/err'] = (train['malign_penalty']+1)/(train['err_malign']+1)

test['penalty-err'] = test['malign_penalty'] - test['err_malign']
test['d_size-ratio'] = test['damage_size'] - test['damage_ratio']
test['d_ratio-m_ratio'] = test['damage_ratio'] - (test['malign_ratio'])
test['penalty/std'] = test['malign_penalty']/test['std_dev_malign']
test['mass/area'] = (test['mass_npea'])/(test['exposed_area'])
test['area/mass'] = test['exposed_area']/test['mass_npea']
test['err/std'] = test['penalty-err']/test['std_dev_malign']
test['dsr/ps'] = test['damage_size']/test['penalty-err']
test['std/area'] = test['std_dev_malign']/test['exposed_area']
test['err/area'] = test['err_malign']/test['exposed_area']
test['dr/area'] = (test['damage_ratio']*100)/test['exposed_area']
test['std/err'] = (test['std_dev_malign']+1)/(test['err_malign']+1)
test['penalty/err'] = (test['malign_penalty']+1)/(test['err_malign']+1)

In [6]:
train.columns

Index(['mass_npea', 'size_npear', 'malign_ratio', 'damage_size',
       'exposed_area', 'std_dev_malign', 'err_malign', 'malign_penalty',
       'damage_ratio', 'penalty-err', 'd_size-ratio', 'd_ratio-m_ratio',
       'penalty/std', 'mass/area', 'area/mass', 'err/std', 'dsr/ps',
       'std/area', 'err/area', 'dr/area', 'std/err', 'penalty/err'],
      dtype='object')

In [7]:
test.columns

Index(['mass_npea', 'size_npear', 'malign_ratio', 'damage_size',
       'exposed_area', 'std_dev_malign', 'err_malign', 'malign_penalty',
       'damage_ratio', 'penalty-err', 'd_size-ratio', 'd_ratio-m_ratio',
       'penalty/std', 'mass/area', 'area/mass', 'err/std', 'dsr/ps',
       'std/area', 'err/area', 'dr/area', 'std/err', 'penalty/err'],
      dtype='object')

In [8]:
# transformation

train['exposed_area'] = np.log1p(train['exposed_area'])

test['exposed_area'] = np.log1p(test['exposed_area'])

In [9]:
#defining metric

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse = make_scorer(RMSE, greater_is_better=False)

In [10]:
#Hyperparameter tuning of extratrees Regressor using Bayesian optimization

def etc_cv(n_estimators, min_samples_split, max_features, data, targets):
    estimator = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2,
        n_jobs = -1
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [11]:
def optimize_etc(data, targets):
    def etc_crossval(n_estimators, min_samples_split, max_features):
        return etc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=etc_crossval,
        pbounds={
            "n_estimators": (100, 400),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=15, init_points=10)

    print("Final result:", optimizer.max)

In [12]:
print(Colours.yellow("--- Optimizing Extra Trees ---"))
optimize_etc(train, target)

--- Optimizing Extra Trees ---
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -3.953    |  0.2722   |  16.31    |  231.3    |
|  2        | -3.906    |  0.806    |  19.94    |  181.8    |
|  3        | -3.97     |  0.3485   |  20.44    |  387.4    |
|  4        | -3.767    |  0.8875   |  10.23    |  250.3    |
|  5        | -3.893    |  0.7144   |  18.39    |  211.1    |
|  6        | -3.824    |  0.6045   |  13.57    |  104.1    |
|  7        | -3.944    |  0.7948   |  22.3     |  209.5    |
|  8        | -3.668    |  0.6532   |  3.734    |  210.6    |
|  9        | -3.859    |  0.9389   |  16.98    |  219.2    |
|  10       | -3.747    |  0.8091   |  9.287    |  270.4    |
|  11       | -3.667    |  0.999    |  2.0      |  334.9    |
|  12       | -3.668    |  0.999    |  2.0      |  142.0    |
|  13       | -3.963    |  0.999    |  25.0     |  325.9    |
|  14       | -3.968    |  0.999    |  

In [14]:
etc = ExtraTreesRegressor(n_estimators=int(366.84350004540295),
                         min_samples_split=int(2.0),
                         max_features=0.999,
                         n_jobs = -1,
                         random_state = 2)

In [24]:
#Hyperparameter tuning of RandomForest Regressor using Bayesian optimization

def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    estimator = RandomForestRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2,
        n_jobs = -1
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [26]:
def optimize_rfc(data, targets):
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        return rfc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (100, 450),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=15, init_points=10)

    print("Final result:", optimizer.max)

In [27]:
print(Colours.green("--- Optimizing Random Forest ---"))
optimize_rfc(train, target)

--- Optimizing Random Forest ---
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -3.945    |  0.2722   |  16.31    |  253.2    |
|  2        | -3.996    |  0.806    |  19.94    |  195.4    |
|  3        | -3.975    |  0.3485   |  20.44    |  435.3    |
|  4        | -3.917    |  0.8875   |  10.23    |  275.3    |
|  5        | -3.979    |  0.7144   |  18.39    |  229.6    |
|  6        | -3.929    |  0.6045   |  13.57    |  104.8    |
|  7        | -4.01     |  0.7948   |  22.3     |  227.7    |
|  8        | -3.845    |  0.6532   |  3.734    |  229.1    |
|  9        | -3.976    |  0.9389   |  16.98    |  239.0    |
|  10       | -3.897    |  0.8091   |  9.287    |  298.8    |
|  11       | -3.833    |  0.1      |  2.0      |  377.9    |
|  12       | -3.854    |  0.8123   |  4.699    |  377.9    |
|  13       | -3.812    |  0.2212   |  2.083    |  143.6    |
|  14       | -3.855    |  0.8866   |

In [15]:
rfc = RandomForestRegressor(n_estimators = int(309.9622556931372),
                           min_samples_split=int(2.1527588902394075),
                           max_features = 0.18433272597322473,
                           n_jobs = -1, 
                           random_state = 2)

In [34]:
#Hyperparameter tuning of LightGBM Regressor using Bayesian optimization

def lgb_cv(n_estimators, num_leaves, min_child_samples, subsample, data, targets):
    estimator = LGBMRegressor(
        n_estimators=n_estimators,
        num_leaves = num_leaves,
        min_child_samples=min_child_samples,
        subsample = subsample,
        random_state = 2
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [35]:
def optimize_lgb(data, targets):
    def lgb_crossval(n_estimators, num_leaves, min_child_samples, subsample):
        return lgb_cv(
            n_estimators=int(n_estimators),
            num_leaves = int(num_leaves),
            min_child_samples=int(min_child_samples),
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=lgb_crossval,
        pbounds={
            "n_estimators": (200,500),
            "num_leaves": (30,80),
            "min_child_samples": (5,30),
            "subsample": (0.6,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [36]:
print(Colours.blue("--- Optimizing Light GBM ---"))
optimize_lgb(train, target)

--- Optimizing Light GBM ---
|   iter    |  target   | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------
|  1        | -3.855    |  9.788    |  386.6    |  51.89    |  0.9141   |
|  2        | -3.89     |  24.5     |  281.8    |  43.82    |  0.9207   |
|  3        | -3.857    |  28.95    |  462.8    |  47.89    |  0.8004   |
|  4        | -3.848    |  22.09    |  413.8    |  48.51    |  0.8245   |
|  5        | -3.854    |  17.58    |  204.1    |  68.64    |  0.9531   |
|  6        | -3.876    |  14.12    |  384.6    |  33.77    |  0.7475   |
|  7        | -3.879    |  28.33    |  395.4    |  49.86    |  0.9155   |
|  8        | -3.838    |  12.92    |  370.4    |  73.46    |  0.7745   |
|  9        | -3.859    |  25.05    |  243.1    |  65.21    |  0.8818   |
|  10       | -3.831    |  10.47    |  477.5    |  52.11    |  0.9637   |
|  11       | -3.927    |  6.495    |  255.3    |  32.37    |  0.87     |
|  12    

In [16]:
lgb = LGBMRegressor(
        n_estimators=int(478.3739357476376),
        num_leaves = int(79.29868320714608),
        min_child_samples=int(13.577585500929393),
        subsample = 1.0,
        random_state = 2
    )

In [39]:
#Hyperparameter tuning of XGBoost Regressor using Bayesian optimization

def xgb_cv(n_estimators, max_depth, gamma, min_child_weight, subsample, data, targets):
    estimator = XGBRegressor(
        n_estimators=n_estimators,
        max_depth = max_depth,
        gamma = gamma,
        min_child_weight=min_child_weight,
        subsample = subsample,
        random_state = 2,
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [40]:
def optimize_xgb(data, targets):
    def xgb_crossval(n_estimators, max_depth, gamma, min_child_weight, subsample):
        return xgb_cv(
            n_estimators=int(n_estimators),
            max_depth = int(max_depth),
            gamma = gamma,
            min_child_weight=min_child_weight,
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        pbounds={
            "n_estimators": (200, 500),
            "max_depth": (6,15),
            "gamma": (0,10),
            "min_child_weight": (0,10),
            "subsample": (0.8,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [41]:
print(Colours.red("--- Optimizing XGBoost ---"))
optimize_xgb(train, target)

--- Optimizing XGBoost ---
|   iter    |  target   |   gamma   | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        | -4.058    |  1.915    |  11.6     |  4.377    |  435.6    |  0.956    |
|  2        | -4.044    |  2.726    |  8.488    |  8.019    |  487.4    |  0.9752   |
|  3        | -4.085    |  3.578    |  10.51    |  6.835    |  413.8    |  0.8741   |
|  4        | -4.074    |  5.612    |  10.53    |  0.1377   |  431.8    |  0.9765   |
|  5        | -4.077    |  3.649    |  11.54    |  0.7538   |  310.6    |  0.9866   |
|  6        | -4.095    |  6.514    |  9.575    |  7.887    |  295.1    |  0.9136   |
|  7        | -4.078    |  8.691    |  9.926    |  8.021    |  243.1    |  0.9409   |
|  8        | -4.05     |  7.046    |  7.969    |  9.249    |  332.6    |  0.9819   |
|  9        | -4.108    |  0.5981   |  7.659    |  0.4736   |  402.5    |  0.9189   |
|  10       | -4.143    |  

In [17]:
xgb = XGBRegressor(
        n_estimators=int(370.207778935361),
        max_depth = int(15.0),
        gamma = 0.0,
        min_child_weight=10.0,
        subsample = 1.0,
        random_state = 2,
    )

In [14]:
#Hyperparameter tuning of Catboost Regressor using Bayesian optimization

def cb_cv(n_estimators, depth,data, targets):
    estimator = CatBoostRegressor(
        n_estimators=n_estimators,
#         learning_rate=learning_rate,
        depth=depth,
        random_state = 2,
        verbose = 0,
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [15]:
def optimize_cb(data, targets):
    def cb_crossval(n_estimators, depth):
        return cb_cv(
            n_estimators=int(n_estimators),
#             learning_rate = learning_rate,
            depth = int(depth),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=cb_crossval,
        pbounds={
            "n_estimators": (200, 600),
#             "learning_rate": (0.01,10),
            "depth": (4,16),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [17]:
print(Colours.bold("--- Optimizing Catboost ---"))
optimize_cb(train, target)

In [7]:
cb = CatBoostRegressor(n_estimators=309,
                      depth = 13,
                      verbose = 0,
                      random_state = 2)

In [11]:
#Hyperparameter tuning of XGBoost RandomForest Regressor using Bayesian optimization

def xgb_cv(n_estimators, max_depth, gamma, min_child_weight, subsample, data, targets):
    estimator = XGBRFRegressor(
        n_estimators=n_estimators,
        max_depth = max_depth,
        gamma = gamma,
        min_child_weight=min_child_weight,
        subsample = subsample,
        random_state = 2,
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring=rmse, cv=5)
    return cval.mean()

In [12]:
def optimize_xgb(data, targets):
    def xgb_crossval(n_estimators, max_depth, gamma, min_child_weight, subsample):
        return xgb_cv(
            n_estimators=int(n_estimators),
            max_depth = int(max_depth),
            gamma = gamma,
            min_child_weight=min_child_weight,
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        pbounds={
            "n_estimators": (200, 500),
            "max_depth": (6,15),
            "gamma": (0,10),
            "min_child_weight": (0,10),
            "subsample": (0.8,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [24]:
print(Colours.red("--- Optimizing XGBoost RandomForest ---"))
optimize_xgb(train, target)

In [18]:
xgbrf = XGBRFRegressor(
        n_estimators=int(463.8),
        max_depth = int(15.0),
        gamma = 10.0,
        min_child_weight=0.0,
        subsample = 0.8,
        random_state = 2,
    )

In [19]:
#Stacking least correlated models

estimators = [('etc', etc), ('rfc', rfc), ('xgb', xgb), ('lgb', lgb), ('xgbrf', xgbrf)]

model = StackingRegressor(estimators=estimators)

In [20]:
scores = cross_val_score(model, train, target, cv = 5, scoring = rmse)

scores.mean()

-3.5971750064096297

In [19]:
#Fitting model

model.fit(train, target)

StackingRegressor(cv=None,
                  estimators=[('etc',
                               ExtraTreesRegressor(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=0.999,
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=366, n_jobs=

In [20]:
#Predicting

preds = model.predict(test)

In [21]:
sub = pd.read_csv('/run/media/devesh/Hard Disk/Projects/Machinehack-Melanoma Tumor Size Predicition/input/sample_submission.csv')

In [22]:
sub['tumor_size'] = preds

In [23]:
#Creating submission file

sub.to_csv('ensemble_2.csv', index = False)